## Set up

In [ ]:
!pip install transformers==4.37.0
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 33.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.37.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AdamW
import os
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from datasets import load_dataset

torch.manual_seed(42)
np.random.seed(42)

## Google Drive linking
Make sure you have a CSE_354_Project in your Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
base_dir = "drive/MyDrive/CSE_354_Project/"
%cd $base_dir

Mounted at /content/drive
/content/drive/MyDrive/CSE_354_Project


## Loading the dataset
This make an API request to hugging face then we verify the data

In [ ]:
ds = load_dataset("community-datasets/per_sent")
print(ds)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/14.0k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.84M [00:00<?, ?B/s]

test_random-00000-of-00001.parquet:   0%|          | 0.00/1.60M [00:00<?, ?B/s]

test_fixed-00000-of-00001.parquet:   0%|          | 0.00/2.27M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/3355 [00:00<?, ? examples/s]

Generating test_random split:   0%|          | 0/579 [00:00<?, ? examples/s]

Generating test_fixed split:   0%|          | 0/827 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/578 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['DOCUMENT_INDEX', 'TITLE', 'TARGET_ENTITY', 'DOCUMENT', 'MASKED_DOCUMENT', 'TRUE_SENTIMENT', 'Paragraph0', 'Paragraph1', 'Paragraph2', 'Paragraph3', 'Paragraph4', 'Paragraph5', 'Paragraph6', 'Paragraph7', 'Paragraph8', 'Paragraph9', 'Paragraph10', 'Paragraph11', 'Paragraph12', 'Paragraph13', 'Paragraph14', 'Paragraph15'],
        num_rows: 3355
    })
    test_random: Dataset({
        features: ['DOCUMENT_INDEX', 'TITLE', 'TARGET_ENTITY', 'DOCUMENT', 'MASKED_DOCUMENT', 'TRUE_SENTIMENT', 'Paragraph0', 'Paragraph1', 'Paragraph2', 'Paragraph3', 'Paragraph4', 'Paragraph5', 'Paragraph6', 'Paragraph7', 'Paragraph8', 'Paragraph9', 'Paragraph10', 'Paragraph11', 'Paragraph12', 'Paragraph13', 'Paragraph14', 'Paragraph15'],
        num_rows: 579
    })
    test_fixed: Dataset({
        features: ['DOCUMENT_INDEX', 'TITLE', 'TARGET_ENTITY', 'DOCUMENT', 'MASKED_DOCUMENT', 'TRUE_SENTIMENT', 'Paragraph0', 'Paragraph1', 'Paragraph2', 'Paragraph3',

In [ ]:
train_dataset = load_dataset("community-datasets/per_sent", split="train")
test_random_dataset = load_dataset("community-datasets/per_sent", split="test_random")
test_fixed_dataset = load_dataset("community-datasets/per_sent", split="test_fixed")
validation_dataset = load_dataset("community-datasets/per_sent", split="validation")

In [ ]:
print(train_dataset)
print(test_random_dataset)
print(test_fixed_dataset)
print(validation_dataset)

for row in train_dataset:
  print(row)
  print(row['TARGET_ENTITY'])
  break

Dataset({
    features: ['DOCUMENT_INDEX', 'TITLE', 'TARGET_ENTITY', 'DOCUMENT', 'MASKED_DOCUMENT', 'TRUE_SENTIMENT', 'Paragraph0', 'Paragraph1', 'Paragraph2', 'Paragraph3', 'Paragraph4', 'Paragraph5', 'Paragraph6', 'Paragraph7', 'Paragraph8', 'Paragraph9', 'Paragraph10', 'Paragraph11', 'Paragraph12', 'Paragraph13', 'Paragraph14', 'Paragraph15'],
    num_rows: 3355
})
Dataset({
    features: ['DOCUMENT_INDEX', 'TITLE', 'TARGET_ENTITY', 'DOCUMENT', 'MASKED_DOCUMENT', 'TRUE_SENTIMENT', 'Paragraph0', 'Paragraph1', 'Paragraph2', 'Paragraph3', 'Paragraph4', 'Paragraph5', 'Paragraph6', 'Paragraph7', 'Paragraph8', 'Paragraph9', 'Paragraph10', 'Paragraph11', 'Paragraph12', 'Paragraph13', 'Paragraph14', 'Paragraph15'],
    num_rows: 579
})
Dataset({
    features: ['DOCUMENT_INDEX', 'TITLE', 'TARGET_ENTITY', 'DOCUMENT', 'MASKED_DOCUMENT', 'TRUE_SENTIMENT', 'Paragraph0', 'Paragraph1', 'Paragraph2', 'Paragraph3', 'Paragraph4', 'Paragraph5', 'Paragraph6', 'Paragraph7', 'Paragraph8', 'Paragraph9', '

## Baseline Model Creation
Using DistillBERT with 3 classes, and a special mask_token which is represented by [TGT] in Masked document of the dataset

In [ ]:
class DistillBERT():
  def __init__(self, model_name='distilbert-base-uncased', num_classes=3):
    self.tokenizer = AutoTokenizer.from_pretrained(model_name)
    self.tokenizer.add_special_tokens({'mask_token': '[TGT]'})
    self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)
    self.model.resize_token_embeddings(len(self.tokenizer))


  def get_tokenizer_and_model(self):
    return self.model, self.tokenizer

Using the model's tokenizer to tokenize the input dataset

In [ ]:
class DatasetLoader(Dataset):

  def __init__(self, dataset, tokenizer):
    self.dataset = dataset
    self.tokenizer = tokenizer

  def tokenize_data(self):
    print("Tokenizing data...")
    tokens = []
    labels = []

    for row in tqdm(self.dataset):
      text = row['MASKED_DOCUMENT']
      label = row['TRUE_SENTIMENT']

      tokens.append(torch.tensor(self.tokenizer.encode(text, max_length=512, truncation=True, add_special_tokens=True)))
      labels.append(label)


    tokens = pad_sequence(tokens, batch_first=True)
    labels = torch.tensor(labels)
    dataset = TensorDataset(tokens, labels)
    return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
    processed_dataset = self.tokenize_data()

    data_loader = DataLoader(
        processed_dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )

    return data_loader

The model's fine-tuning stages by giving it the task specific data.

The Optimizer is created in the execution stage, which have the hyperparameters of learning rate and epsilon.

In [ ]:
class Trainer():

  def __init__(self, options):
    self.device = options['device']
    self.train_data = options['train_data']
    self.batch_size = options['batch_size']
    self.epochs = options['epochs']
    self.save_path = options['save_path']
    transformer = DistillBERT()
    self.model, self.tokenizer = transformer.get_tokenizer_and_model()
    self.model.to(self.device)

  def get_performance_metrics(self, preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    precision = precision_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    recall = recall_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    f1 = f1_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    return precision, recall, f1

  def set_training_parameters(self):
    for i, layer in enumerate(self.model.distilbert.transformer.layer):
      for param in layer.parameters():
        param.requires_grad = True

    for param in self.model.classifier.parameters():
      param.requires_grad = True

  def train(self, data_loader, optimizer):
    self.model.train()
    total_recall = 0
    total_precision = 0
    total_f1 = 0
    total_loss = 0

    for batch_idx, (text, labels) in enumerate(tqdm(data_loader)):
      self.model.zero_grad()

      text = text.to(self.device)
      labels = labels.to(self.device)

      outputs = self.model(text, labels=labels)
      predicted = outputs.logits.cpu().detach().numpy()

      loss = outputs.loss
      total_loss += loss.item()
      loss.backward()
      optimizer.step()

      precision, recall, f1 = self.get_performance_metrics(predicted, labels.cpu().numpy())
      total_precision += precision
      total_recall += recall
      total_f1 += f1

    precision = total_precision/len(data_loader)
    recall = total_recall/len(data_loader)
    f1 = total_f1/len(data_loader)
    loss = total_loss/len(data_loader)

    return precision, recall, f1, loss

  def eval(self, data_loader):
    self.model.eval()
    total_recall = 0
    total_precision = 0
    total_f1 = 0
    total_loss = 0

    with torch.no_grad():
      for batch_idx, (text, labels) in enumerate(tqdm(data_loader)):
        text = text.to(self.device)
        labels = labels.to(self.device)

        outputs = self.model(text, labels=labels)
        predicted = outputs.logits.cpu().detach().numpy()

        loss = outputs.loss
        total_loss += loss.item()

        precision, recall, f1 = self.get_performance_metrics(predicted, labels.cpu().numpy())
        total_precision += precision
        total_recall += recall
        total_f1 += f1

      precision = total_precision/len(data_loader)
      recall = total_recall/len(data_loader)
      f1 = total_f1/len(data_loader)
      loss = total_loss/len(data_loader)

      return precision, recall, f1, loss

  def save_transformer(self):
    self.model.save_pretrained(self.save_path)
    self.tokenizer.save_pretrained(self.save_path)

  def execute(self):
    last_best = 0
    train_dataset = DatasetLoader(self.train_data, self.tokenizer)
    train_data_loader = train_dataset.get_data_loaders(self.batch_size)

    val_dataset = DatasetLoader(validation_dataset, self.tokenizer)
    val_data_loader = val_dataset.get_data_loaders(self.batch_size)

    optimizer = torch.optim.AdamW(self.model.parameters(), lr = 3e-5, eps = 1e-8)
    self.set_training_parameters()

    for epoch_i in range(0, self.epochs):
      train_precision, train_recall, train_f1, train_loss = self.train(train_data_loader, optimizer)
      print(f'Epoch {epoch_i + 1}: train_loss: {train_loss:.4f} train_precision: {train_precision:.4f} train_recall: {train_recall:.4f} train_f1: {train_f1:.4f}')
      val_precision, val_recall, val_f1, val_loss = self.eval(val_data_loader)
      print(f'Epoch {epoch_i + 1}: val_loss: {val_loss:.4f} val_precision: {val_precision:.4f} val_recall: {val_recall:.4f} val_f1: {val_f1:.4f}')

      if val_f1 > last_best:
        print("Saving model..")
        self.save_transformer()
        last_best = val_f1
        print("Model saved.")

Hyper Params of
1. Batch size
2. Epochs

Then the additional of save path of the model which is set to *distilbert_per_sent_baseline* at the moment

In [ ]:
BATCH_SIZE = 16
EPOCHS = 3

SAVE_PATH = 'distilbert_per_sent_baseline'

## Baseline Training
training on the train_dataset we've loaded up previously, then verfying it with the validation_dataset each epochs to see the improvement of the model

In [ ]:
options = {}
options['device'] = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
options['train_data'] = train_dataset
options['batch_size'] = BATCH_SIZE
options['epochs'] = EPOCHS
options['save_path'] = SAVE_PATH

trainer = Trainer(options)
trainer.execute()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Epoch 2: val_loss: 0.8694 val_precision: 0.5858 val_recall: 0.5760 val_f1: 0.5439


100%|██████████| 210/210 [02:46<00:00,  1.26it/s]


Epoch 3: train_loss: 0.7192 train_precision: 0.6709 train_recall: 0.6720 train_f1: 0.6489


100%|██████████| 37/37 [00:09<00:00,  3.72it/s]


Epoch 3: val_loss: 0.9075 val_precision: 0.6223 val_recall: 0.5777 val_f1: 0.5762
Saving model..
Model saved.


## Baseline Testing
Verifying the model performance with fresh data from testing split.

In [ ]:
class Tester:
    def __init__(self, options):
        self.device = options['device']
        self.test_data = options['test_data']
        self.batch_size = options['batch_size']
        self.save_path = options['save_path']

        transformer = DistillBERT()
        self.model, self.tokenizer = transformer.get_tokenizer_and_model()
        self.model.to(self.device)

    def load_transformer(self):
        self.model = AutoModelForSequenceClassification.from_pretrained(self.save_path)
        self.tokenizer = AutoTokenizer.from_pretrained(self.save_path)
        self.model.to(self.device)
        self.model.eval()

    def get_performance_metrics(self, preds, labels):
        pred_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()
        precision = precision_score(labels_flat, pred_flat, zero_division=0, average='weighted')
        recall = recall_score(labels_flat, pred_flat, zero_division=0, average='weighted')
        f1 = f1_score(labels_flat, pred_flat, zero_division=0, average='weighted')
        return precision, recall, f1

    def test(self, data_loader):
        self.model.eval()
        total_recall = 0
        total_precision = 0
        total_f1 = 0
        total_loss = 0

        with torch.no_grad():
            for batch_idx, (text, labels) in enumerate(tqdm(data_loader)):
                text = text.to(self.device)
                labels = labels.to(self.device)

                outputs = self.model(text, labels=labels)
                predicted = outputs.logits.cpu().detach().numpy()
                loss = outputs.loss
                total_loss += loss.item()

                precision, recall, f1 = self.get_performance_metrics(predicted, labels.cpu().numpy())
                total_precision += precision
                total_recall += recall
                total_f1 += f1

            precision = total_precision / len(data_loader)
            recall = total_recall / len(data_loader)
            f1 = total_f1 / len(data_loader)
            loss = total_loss / len(data_loader)

            return precision, recall, f1, loss

    def execute(self):
        self.load_transformer()
        test_dataset = DatasetLoader(self.test_data, self.tokenizer)
        test_data_loader = test_dataset.get_data_loaders(self.batch_size)
        precision, recall, f1, loss = self.test(test_data_loader)
        print(f'precision: {precision:.4f} recall: {recall:.4f} f1: {f1:.4f}')
        print(f'Test loss: {loss:.4f}')
        y_true = []
        y_pred = []
        with torch.no_grad():
            for batch_idx, (text, labels) in enumerate(tqdm(test_data_loader)):
                text = text.to(self.device)
                labels = labels.to(self.device)
                outputs = self.model(text)
                predicted = torch.argmax(outputs.logits, dim=1).cpu().numpy()
                y_true.extend(labels.cpu().numpy())
                y_pred.extend(predicted)

        report = classification_report(y_true, y_pred, target_names=["Negative", "Neutral", "Positive"])
        print()
        print(report)


Testing on the test_random_dataset (random and different main target entity)

In [ ]:
options={}
options['device'] = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
options['test_data'] = test_random_dataset
options['batch_size'] = BATCH_SIZE
options['save_path'] = SAVE_PATH

tester = Tester(options)
tester.execute()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizing data...


100%|██████████| 37/37 [00:10<00:00,  3.69it/s]


precision: 0.5998 recall: 0.5552 f1: 0.5515
Test loss: 0.9371


100%|██████████| 37/37 [00:09<00:00,  3.73it/s]


              precision    recall  f1-score   support

    Negative       0.32      0.44      0.37        73
     Neutral       0.48      0.39      0.43       213
    Positive       0.68      0.70      0.69       293

    accuracy                           0.55       579
   macro avg       0.49      0.51      0.50       579
weighted avg       0.56      0.55      0.55       579



Testing on the test_fixed_dataset (stuff have repeat high frequence main target entity)

In [ ]:
options={}
options['device'] = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
options['test_data'] = test_fixed_dataset
options['batch_size'] = BATCH_SIZE
options['save_path'] = SAVE_PATH

tester = Tester(options)
tester.execute()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizing data...


100%|██████████| 52/52 [00:14<00:00,  3.57it/s]


precision: 0.4933 recall: 0.4763 f1: 0.4671
Test loss: 1.1193


100%|██████████| 52/52 [00:14<00:00,  3.60it/s]


              precision    recall  f1-score   support

    Negative       0.33      0.26      0.29       139
     Neutral       0.45      0.38      0.41       320
    Positive       0.53      0.64      0.58       368

    accuracy                           0.48       827
   macro avg       0.44      0.43      0.43       827
weighted avg       0.46      0.48      0.47       827



---
## Idea 1 Mask TGT Token for Classification
Below are the implementation for idea 1